In [ ]:
pip install tensorflow-model-optimization

In [ ]:
import tensorflow as tf
import numpy as np
import struct
from scipy.sparse import csr_matrix

def prune_model(model_type):
    if model_type == "generator":
        original_path = '/content/drive/MyDrive/DCGANmodels/finetuneC1/finaC1LungandColonCancerGenerator.h5'
        pruned_model_path = '/content/drive/MyDrive/DCGANmodels/pruned/LungandColonCancerGenatatorPrunedModel.h5'
    elif model_type == "discriminator":
        original_path = '/content/drive/MyDrive/DCGANmodels/finetuneC1/finalC1LungandColonCancerDiscriminator.h5'
        pruned_model_path = '/content/drive/MyDrive/DCGANmodels/pruned/LungandColonCancerDiscriminatorPrunedModel.h5'
    else:
        raise ValueError("Invalid model type. Choose 'Generator' or 'Discriminator'.")

    original_model = tf.keras.models.load_model(original_path)

    pruning_threshold = 0.37

    pruned_model = tf.keras.models.clone_model(original_model)

    for layer in pruned_model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
            weights = layer.get_weights()
            pruned_weights = []
            for w in weights:
                threshold = np.percentile(np.abs(w), pruning_threshold * 100)
                pruned_w = np.where(np.abs(w) < threshold, 0.0, w)
                pruned_weights.append(pruned_w)
            layer.set_weights(pruned_weights)

    total_param_count = original_model.count_params()
    pruned_param_count = sum(np.prod(w.shape) for w in pruned_model.get_weights())
    sparsity_percentage = np.mean([np.count_nonzero(w == 0) / w.size for w in pruned_weights]) * 100

    # print("Number of parameters in the original model:", total_param_count)
    # print("Number of parameters in the pruned model:", pruned_param_count)
    # print("Sparsity percentage of the pruned model's weights:", sparsity_percentage)

    pruned_model = tf.keras.models.load_model(pruned_model_path, compile=False)
    pruned_model.compile()

    return pruned_model

In [ ]:
# import numpy as np

# array_3d = np.array([
#     [[ 0.06715256,  0.02249832,  0.02895072, ...,  0.04440202, -0.01655976, -0.04388078],
#      [ 0.03473309, -0.06043807,  0.01735245, ..., -0.02202907, -0.06448454,  0.01639612],
#      [-0.0473573 ,  0.01233201, -0.02072671, ...,  0.03140628, -0.0041892 ,  0.07093197]],

#     [[ 0.02187997,  0.03086492,  0.04409173, ...,  0.05337322,  0.01359168,  0.04372563],
#      [-0.05038473, -0.001738  ,  0.00395834, ..., -0.03038783,  0.06745648, -0.05602755],
#      [-0.05578252,  0.01507156, -0.03894692, ...,  0.03607336,  0.02079662, -0.04610587]],

#     [[-0.04035418,  0.06329109,  0.06215158, ...,  0.01353917,  0.0293226 ,  0.02698114],
#      [-0.04121484,  0.05496372, -0.05980618, ..., -0.00269137, -0.04137776,  0.01189379],
#      [-0.07361974,  0.01781902, -0.00220644, ...,  0.05036451,  0.00818687,  0.01042236]],

#     [[ 0.0537307 , -0.03556773, -0.00050614, ..., -0.05648738, -0.06677028,  0.04372723],
#      [-0.03774463,  0.00601193,  0.01931232, ...,  0.06274822, -0.02565265, -0.03993678],
#      [-0.04469112, -0.06838983, -0.01179312, ..., -0.07440331,  0.07464144,  0.01879468]]
# ])
# element = array_3d[3, 2, 1]
# print(element)

In [ ]:
def identify_zero_neurons(model, threshold, pruned_param_count_required=False):
    pruned_param_count = 0
    pruned_weights = []

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
            weights = layer.get_weights()
            pruned_w = [np.where(np.abs(w) < threshold * np.max(np.abs(w)), 0.0, w) for w in weights]
            pruned_weights.extend(pruned_w)
            pruned_param_count += np.sum([np.count_nonzero(w) for w in pruned_w])
        else:
            pruned_weights.extend(layer.get_weights())

    model.set_weights(pruned_weights)

    total_param_count = model.count_params()
    sparsity_percentage = (1 - pruned_param_count / total_param_count) * 100

    print("Number of parameters in the original model:", total_param_count)
    print("Number of parameters in the pruned model:", pruned_param_count)
    print("Sparsity percentage of the pruned model's weights:", sparsity_percentage)

    return pruned_param_count if pruned_param_count_required else total_param_count

In [ ]:
def get_sparse_representation(weights):
    sparse_weights = []
    for weight_matrix in weights:
        sparse_matrix = csr_matrix(weight_matrix)
        sparse_weights.append((sparse_matrix.data, sparse_matrix.indices, sparse_matrix.indptr))
    return sparse_weights

In [ ]:
def extract_weights_from_interpreter(interpreter):
    weights = []
    for tensor_details in interpreter.get_tensor_details():
        if "weight" in tensor_details['name']:
            tensor_data = interpreter.tensor(tensor_details['index'])()
            weights.append(tensor_data)
    return weights

In [ ]:
def rle_encode(data):
    encoding = []
    i = 0
    while i < len(data):
        count = 1
        while i + 1 < len(data) and data[i] == data[i + 1]:
            count += 1
            i += 1
        encoding.append((data[i], count))
        i += 1
    return encoding

In [ ]:
def custom_compress(sparse_weights):
    compressed_weights = []
    for data, indices, indptr in sparse_weights:
        encoded_indices = rle_encode(indices)
        encoded_data = np.diff(data, prepend=data[0])
        compressed_weights.append((encoded_indices, encoded_data, indptr))
    return compressed_weights

In [ ]:
def save_compressed_weights(file_path, compressed_weights):
    with open(file_path, 'wb') as f:
        for encoded_indices, encoded_data, indptr in compressed_weights:
            f.write(struct.pack('I', len(encoded_indices)))
            f.write(struct.pack(f'{len(encoded_indices)}I', *encoded_indices))
            f.write(struct.pack('I', len(encoded_data)))
            f.write(struct.pack(f'{len(encoded_data)}f', *encoded_data))
            f.write(struct.pack('I', len(indptr)))
            f.write(struct.pack(f'{len(indptr)}I', *indptr))

In [ ]:
def custom_decompress(file_path):
    decompressed_weights = []
    with open(file_path, 'rb') as f:
        while True:
            try:
                indices_len = struct.unpack('I', f.read(4))[0]
                encoded_indices = struct.unpack(f'{indices_len}I', f.read(4 * indices_len))
                data_len = struct.unpack('I', f.read(4))[0]
                encoded_data = struct.unpack(f'{data_len}f', f.read(4 * data_len))
                indptr_len = struct.unpack('I', f.read(4))[0]
                indptr = struct.unpack(f'{indptr_len}I', f.read(4 * indptr_len))

                indices = np.cumsum(encoded_indices)
                data = np.cumsum(encoded_data)
                decompressed_weights.append((data, indices, indptr))
            except:
                break
    return decompressed_weights

In [ ]:
def set_sparse_weights(model, decompressed_weights):
    for layer, (data, indices, indptr) in zip(model.layers, decompressed_weights):
        if len(layer.get_weights()) > 0:
            weight_matrix = csr_matrix((data, indices, indptr), shape=layer.get_weights()[0].shape).toarray()
            layer.set_weights([weight_matrix])

In [ ]:
import random
import time

class NoCRouter:
    def __init__(self, pe_array):
        self.pe_array = pe_array
        self.reconfig_executor = ReconfigurationExecutor()

    def broadcast(self, source_data):
        print("Firing Broadcast Dataflow")
        for z in range(2):
            for y in range(2):
                for x in range(2):
                    print(f"Layer {z}: Data -> PE{y}{x}")
                    self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])

    def unicast(self, source_data, dest_pe):
        print("Firing Unicast Dataflow")
        y, x, z = dest_pe
        print(f"Layer {z}: Data -> PE{y}{x}")
        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])

    def grouped_multicast(self, source_data, row):
        print("Firing Grouped Multicast Dataflow")
        for z in range(2):
            for x in range(2):
                print(f"Layer {z}: Data -> PE{row}{x}")
                self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(row, x, z)])

    def interleaved_multicast(self, source_data):
        print("Firing Interleaved Multicast Dataflow")
        for z in range(2):
            for y in range(2):
                for x in range(2):
                    if (x + y) % 2 == 0:
                        print(f"Layer {z}: Data -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])

    def systolic_1d(self, source_data, row):
        print("Firing Systolic 1D Dataflow")
        for z in range(2):
            for x in range(2):
                if x == 0:
                    print(f"Layer {z}: Data -> PE{row}{x}")
                else:
                    print(f"Layer {z}: PE{row}{x-1} -> PE{row}{x}")
                self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(row, x, z)])

    def systolic_2d(self, source_data):
        print("Firing Systolic 2D Dataflow")
        for z in range(2):
            for y in range(2):
                for x in range(2):
                    if x == 0 and y == 0:
                        print(f"Layer {z}: Data -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])
                    elif x == 0:
                        print(f"Layer {z}: PE{y-1}{x} -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])
                    elif y == 0:
                        print(f"Layer {z}: PE{y}{x-1} -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])
                    else:
                        print(f"Layer {z}: PE{y-1}{x} -> PE{y}{x}")
                        print(f"Layer {z}: PE{y}{x-1} -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])

    def systolic_3d(self, source_data):
        print("Firing Systolic 3D Dataflow")
        for z in range(2):
            for y in range(2):
                for x in range(2):
                    if x == 0 and y == 0 and z == 0:
                        print(f"Layer {z}: Input Data -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])
                    elif x == 0 and y == 0:
                        print(f"Layer {z}: PE{y}{x} -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])
                    elif x == 0:
                        print(f"Layer {z}: PE{y-1}{x} -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])
                    elif y == 0:
                        print(f"Layer {z}: PE{y}{x-1} -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])
                    else:
                        print(f"Layer {z}: PE{y-1}{x} -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])
                        print(f"Layer {z}: PE{y}{x-1} -> PE{y}{x}")
                        self.reconfig_executor.activate_deactivate_pes(activate=True, indices=[(y, x, z)])

    def parallel_systolic_2d(self, source_data):
        print("Firing Parallel Systolic 2D Dataflow")
        for row in range(2):
            self.systolic_2d(source_data)

    def parallel_systolic_1d(self, source_data):
        print("Firing Parallel Systolic 1D Dataflow")
        for row in range(2):
            self.systolic_1d(source_data, row)

class ResourceMonitor:
    def __init__(self, total_params):
        self.computation_load = 0
        self.memory_access_patterns = {}
        self.communication_traffic = 0
        self.total_params = total_params

    def collect_data(self):
        self.computation_load = self.get_computation_load()
        self.memory_access_patterns = self.get_memory_access_patterns()
        self.communication_traffic = self.get_communication_traffic()
        print("Data collected: Computation load =", self.computation_load, "\nMemory access patterns =", self.memory_access_patterns, ", Communication traffic =", self.communication_traffic)

    def get_computation_load(self):
        return random.randint(0, self.total_params)

    def get_memory_access_patterns(self):
        return 'highly_sparse' if random.random() > 0.35 else 'dense'

    def get_communication_traffic(self):
        return random.randint(0, self.total_params)

class DecisionMaker:
    def __init__(self, threshold):
        self.threshold = threshold
        self.reconfig_executor = ReconfigurationExecutor()
        self.noc_router = None  # This will be set from the DynamicReconfigurationSystem

    def set_noc_router(self, noc_router):
        self.noc_router = noc_router

    def make_decision(self, monitor):
        if monitor.computation_load > self.threshold:
            self.adjust_smart_buffers(monitor.computation_load)
        if monitor.communication_traffic > self.threshold:
            self.reallocate_bandwidth(monitor)
        if monitor.memory_access_patterns == 'highly_sparse':
            self.deactivate_specific_pes()

    def adjust_smart_buffers(self, computation_load):
        if computation_load > self.threshold:
            print("Increasing buffer sizes to handle high computation load.")
            self.reconfig_executor.resize_smart_buffers(32)
        else:
            print("Decreasing buffer sizes for efficiency.")
            self.reconfig_executor.resize_smart_buffers(16)

    def reallocate_bandwidth(self, monitor):
        if monitor.communication_traffic > self.threshold:
            if monitor.memory_access_patterns == 'highly_sparse':
                print("Reallocating bandwidth: Waiting in pipeline due to memory access bandwidth issue.")
                # Call both multicast and unicast functions with delay
                self.noc_router.grouped_multicast("source_data", row=0)  # Example row
                time.sleep(1)  # Simulate delay
                self.noc_router.unicast("source_data", dest_pe=(1, 1, 1))  # Example PE
            else:
                print("Reallocating bandwidth: Using idle or deactivated PEs for PE access bandwidth issue.")
                # Fetch the last set of deactivated PEs
                deactivated_pes = self.get_deactivated_pes()
                if self.is_complete_layer_deactivated(deactivated_pes):
                    print("One complete PE layer is deactivated.")
                    self.noc_router.systolic_2d("source_data")
                elif self.is_multiple_layers_deactivated(deactivated_pes):
                    print("More than one complete PE layer is deactivated.")
                    self.noc_router.systolic_3d("source_data")
                elif self.is_one_or_more_rows_deactivated(deactivated_pes):
                    print("One or more rows in a PE are deactivated.")
                    self.noc_router.grouped_multicast("source_data", row=0)  # Example row
                else:
                    print("Only one or more random PEs are deactivated.")
                    self.noc_router.unicast("source_data", dest_pe=deactivated_pes[0])  # Example PE
        else:
            print("Bandwidth allocation is optimal.")
            self.noc_router.systolic_3d("source_data")

    def deactivate_specific_pes(self):
        print("Deactivating specific PEs based on memory access patterns.")
        # Example logic to deactivate every alternate PE
        pes_to_deactivate = [(y, x, z) for z in range(2) for y in range(2) for x in range(2) if (x + y + z) % 2 != 0]
        self.reconfig_executor.activate_deactivate_pes(activate=False, indices=pes_to_deactivate)

    def get_deactivated_pes(self):
        # Placeholder function to return a list of deactivated PEs
        # In real implementation, this should track the actual deactivated PEs
        return [(y, x, z) for z in range(8) for y in range(8) for x in range(8) if (x + y + z) % 2 != 0]

    def is_complete_layer_deactivated(self, deactivated_pes):
        # Check if any complete layer is deactivated
        for z in range(2):
            if all((y, x, z) in deactivated_pes for y in range(2) for x in range(2)):
                return True
        return False

    def is_multiple_layers_deactivated(self, deactivated_pes):
        # Check if more than one complete layer is deactivated
        deactivated_layers = 0
        for z in range(2):
            if all((y, x, z) in deactivated_pes for y in range(2) for x in range(2)):
                deactivated_layers += 1
        return deactivated_layers > 1

    def is_one_or_more_rows_deactivated(self, deactivated_pes):
        # Check if one or more rows in any PE are deactivated
        for z in range(2):
            for y in range(2):
                if all((y, x, z) in deactivated_pes for x in range(8)):
                    return True
        return False

class ReconfigurationExecutor:
    def execute_reconfiguration(self, size=32):
        self.resize_smart_buffers(size)
        self.reconfigure_noc()
        self.activate_deactivate_pes(activate=True, indices=[])
        print("Executed reconfiguration actions.")

    def resize_smart_buffers(self, size=32):
      if size >= 32:
        print(f"Resizing smart buffers to {size} bits.")

    def reconfigure_noc(self):
        print("Reconfiguring NoC based on decision.")

    def activate_deactivate_pes(self, activate=True, indices=[]):
        for index in indices:
            if activate:
                print(f"Activating PE at index: {index}")
            else:
                print(f"Dectivating PE at index: {index}")

class DynamicReconfigurationSystem:
    def __init__(self, pe_array, total_params, threshold):
        self.monitor = ResourceMonitor(total_params)
        self.decision_maker = DecisionMaker(threshold)
        self.reconfiguration_executor = ReconfigurationExecutor()
        self.noc_router = NoCRouter(pe_array)
        self.decision_maker.set_noc_router(self.noc_router)

    def run(self):
        while True:
            self.monitor.collect_data()
            self.decision_maker.make_decision(self.monitor)
            self.reconfiguration_executor.execute_reconfiguration(self.monitor.computation_load)
            self.simulate_dataflows()
            break

    def simulate_dataflows(self):
        source_data = "data_location"
        self.noc_router.broadcast(source_data)
        self.noc_router.unicast(source_data, (1, 1, 1))
        self.noc_router.grouped_multicast(source_data, 1)
        self.noc_router.interleaved_multicast(source_data)
        self.noc_router.systolic_1d(source_data, 0)
        self.noc_router.systolic_2d(source_data)
        self.noc_router.systolic_3d(source_data)
        self.noc_router.parallel_systolic_2d(source_data)
        self.noc_router.parallel_systolic_1d(source_data)

In [ ]:
import tensorflow as tf
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module='tensorflow')

def main():
    model_type = "generator"  # or "discriminator"
    pruned_model = prune_model(model_type)
    threshold = 0.37

    # Quantize the pruned model
    converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    quantized_model = converter.convert()

    # Set up TensorFlow Lite interpreter
    interpreter = tf.lite.Interpreter(model_content=quantized_model)
    interpreter.allocate_tensors()

    # Extract sparse representations
    quantized_weights = extract_weights_from_interpreter(interpreter)
    sparse_weights = get_sparse_representation(quantized_weights)

    # Compress the sparse weights
    compressed_weights = custom_compress(sparse_weights)

    # Save the compressed weights
    compressed_weights_file = f'compressed_{model_type.lower()}_weights.bin'
    save_compressed_weights(compressed_weights_file, compressed_weights)

    # Decompress the weights
    decompressed_weights = custom_decompress(compressed_weights_file)

    # Set the decompressed weights back to the model
    set_sparse_weights(pruned_model, decompressed_weights)

    # Calculate the number of zero neurons and set the threshold
    total_params = identify_zero_neurons(pruned_model, threshold=0.1, pruned_param_count_required=False)
    threshold = total_params // 2

    # Initialize the PE array
    # pe_array = [[[f'PE{y}{x}{z}' for x in range(8)] for y in range(8)] for z in range(8)]
    pe_array = [[[f'PE{y}{x}{z}' for x in range(2)] for y in range(2)] for z in range(2)]

    # Initialize and run the dynamic reconfiguration system
    dynamic_reconfig_system = DynamicReconfigurationSystem(pe_array, total_params, threshold)
    dynamic_reconfig_system.run()

if __name__ == "__main__":
    main()

Number of parameters in the original model: 7082752
Number of parameters in the pruned model: 4378428
Sparsity percentage of the pruned model's weights: 38.181825369573865
Data collected: Computation load = 2547533 
Memory access patterns = highly_sparse , Communication traffic = 3788302
Reallocating bandwidth: Waiting in pipeline due to memory access bandwidth issue.
Firing Grouped Multicast Dataflow
Layer 0: Data -> PE00
Activating PE at index: (0, 0, 0)
Layer 0: Data -> PE01
Activating PE at index: (0, 1, 0)
Layer 1: Data -> PE00
Activating PE at index: (0, 0, 1)
Layer 1: Data -> PE01
Activating PE at index: (0, 1, 1)
Firing Unicast Dataflow
Layer 1: Data -> PE11
Activating PE at index: (1, 1, 1)
Deactivating specific PEs based on memory access patterns.
Dectivating PE at index: (0, 1, 0)
Dectivating PE at index: (1, 0, 0)
Dectivating PE at index: (0, 0, 1)
Dectivating PE at index: (1, 1, 1)
Resizing smart buffers to 2547533 bits.
Reconfiguring NoC based on decision.
Executed reconf